# Pregunta 2

A continuación se importan algunas librerías estándar de ``python`` que serán de utilidad. Luego se definen funciones auxiliares que serán usadas para

In [193]:
# Standard library
from math import ceil
from random import randint, randrange

# Auxiliary functions

# Fast exponentiation
def _fast_exp(a, b):
    if not b:
        return 1
    result = 1
    power = a
    while b:
        if b % 2:
            result = result * power
        b = b // 2
        power = power * power
    return result

# Fast modular exponentiation
def _fast_exp_mod(a, b, n):
    if not b:
        return 1
    if b:
        result = 1
        power = a
        while b:
            if b % 2:
                result = (power * result) % n
            b = b // 2
            power = (power * power) % n
        return result
    return _fast_exp_mod(_mod_inverse(a, n), -b, n)

# Modular inverse
def _mod_inverse(a, n):
    r, s, t = _ext_euclidean(a, n)
    return s % n

# Greatest common divisor
def _gcd(a, b):
    while b:
        temp = b
        b = a % b
        a = temp
    return a

# Extended Euclidean algorithm
def _ext_euclidean(a, b):
    r_0 = a
    s_0 = 1
    t_0 = 0
    r_1 = b
    s_1 = 0
    t_1 = 1
    while r_1:
        r_2 = r_0 % r_1
        s_2 = s_0 - (r_0 // r_1) * s_1
        t_2 = t_0 - (r_0 // r_1) * t_1
        r_0 = r_1
        s_0 = s_1
        t_0 = t_1
        r_1 = r_2
        s_1 = s_2
        t_1 = t_2
    return r_0, s_0, t_0

# Check if n is a power of another number (optimized considering n > 3)
def _is_power(n):
    k = 2
    lim = 4

    # In the first 10^5 primes, all factors smaller than 10^6 already appeared, so we focus on the larger ones
    while lim <= n and k < 52:  # 10^(6 * 52) > 10^309 = max_n
        if _has_integer_root(n, k):
            return True
        k = k + 1
        lim = lim * 2
    return False

# Check if n is a k-th degree power (optimized considering n > 3)
def _has_integer_root(n, k):
    a = 1
    while _fast_exp(a, k) < n:
        a = a * 2
    return _has_integer_root_interval(n, k, a // 2, a)

# Check if n is a k-th degree power inside an interval
def _has_integer_root_interval(n, k, i, j):
    while i <= j:
        if i == j:
            return n == _fast_exp(i, k)
        p = (i + j) // 2
        val = _fast_exp(p, k)
        if n == val:
            return True
        elif val < n:
            i = p + 1
        else:
            j = p - 1
    return False

# Primality test (Solovay – Strassen) with some slight optimizations
def _primality_test(n, k):
    if _is_power(n):
        return False
    neg = 0
    for _ in range(k):
        a = randint(2, n - 1)
        if _gcd(a, n) > 1:
            return False
        b = _fast_exp_mod(a, (n - 1) // 2, n)
        if b == n - 1:
            neg = neg + 1
        elif b != 1:
            return False
    return neg > 0

# Obtain all primes smaller or equal to n (Sieve of Eratosthenes)
def _get_primes(n):
    prime_idx = [1] * (n + 1)
    p = 2
    while (p * p) <= n:
        if prime_idx[p]:
            for i in range(p * p, n + 1, p):
                prime_idx[i] = 0
        p = p + 1
    return [p for p in range(2, n + 1) if prime_idx[p]]

# Generate a pair of unique random prime numbers in the interval [a,b]
def _generate_primes(a, b):
    # Number of repetitions for the primality test
    k = 34  # Probability of error is (1/2)^k

    # Trivial case
    if a == 2:
        return [2, 3]

    # Pre-calculate first 10^5 primes
    first_primes = _get_primes(100000)

    # Search for primes by discarding most candidates in constant time
    primes = []
    while True:
        p = randrange(a + 1, b + 1, 2)  # Only check odd numbers
        if p in primes:  # Both primes need to be different
            continue
        candidate = True
        for n in first_primes:  # Discard with first n prime factors
            if p <= n:
                break
            if not (p % n):
                candidate = False
                break
        if candidate and _primality_test(p, k):  # Test for primality
            primes.append(p)
            if len(primes) == 2:
                return primes

# Generate a random number that is co-prime to n
def _generate_coprime(n):
    # Trivial case
    if n == 2:
        return 1

    # Search for co-primes
    while True:
        c = randint(2, n - 1)
        if _gcd(c, n) == 1:  # GCD must be 1
            return c

A

In [194]:
# RSA

# Receiver
class RSAReceiver:
    def __init__(self, bit_len):
        """
        Arguments:
            bit_len: A lower bound for the number of bits of N,
            the second argument of the public and secret key.
        """
        # Generate random primes P,Q
        p, q = _generate_primes(2 ** (ceil(bit_len / 2) - 1), 2 ** (ceil(bit_len / 2)) - 1)

        # Define N and phi(N)
        n = p * q
        phi_n = n - p - q + 1

        # Generate random co-prime d (to phi(N))
        d = _generate_coprime(phi_n)

        # Get the modular inverse e for d in mod phi(N)
        e = _mod_inverse(d, phi_n)

        # Define private & public keys
        self.private_key = (d, n)
        self.public_key = (e, n)

    # Get public key
    def get_public_key(self):
        """
        Returns:
            public_key: Public key expressed as a Python 'bytearray' using the
            PEM format. This means the public key is divided in:
            (1) The number of bytes of e (4 bytes)
            (2) The number e (as many bytes as indicated in (1))
            (3) The number of bytes of N (4 bytes)
            (4) The number N (as many bytes as indicated in (3))
        """
        e_bytes = ceil(self.public_key[0].bit_length() / 8)
        pem_key = bytearray(e_bytes.to_bytes(4, 'big'))
        pem_key += self.public_key[0].to_bytes(e_bytes, 'big')
        n_bytes = ceil(self.public_key[1].bit_length() / 8)
        pem_key += bytearray(n_bytes.to_bytes(4, 'big'))
        pem_key += self.public_key[1].to_bytes(n_bytes, 'big')
        return pem_key

    # Decrypt the cipher
    def decrypt(self, ciphertext):
        """
        Arguments:
            ciphertext: The ciphertext to decrypt.
        Returns:
            message: The original message.
        """
        # TODO: decrypt blocks using private key
        return ''

# Sender
class RSASender:
    def __init__(self, public_key):
        """
        Arguments:
            public_key: The public key that will be used to encrypt messages.
        """
        self.receiver_key = public_key

    # Encrypt the message
    def encrypt(self, message):
        """
        Arguments:
            message: The plaintext message to encrypt.
        Returns:
            ciphertext: The encrypted message.
        """
        # TODO: encrypt blocks using receiver key
        return ''

A

In [195]:
# Test RSA protocol
if __name__ == '__main__':
    receiver = RSAReceiver(2048)
    print(receiver.get_public_key())

bytearray(b'\x00\x00\x01\x00:\xcc)p\xf8n\xa3\xf2\xb0\xddDE\xca\x16y\xa5\xb8\x86\xd8\n\x1bZ\xdd\xa6>\xefN\xdbP\xff\xd3n\xc03A\xef\xa0\xfd\x18\xaa\x95$\xfb<\xa5\xbc\x1c\xb2Y\x9e\x16+u\xa6\xd7\xc6\xec\x1bn\xb5F\x94\x1an\xd6\xde\xa6J\xbe\xbas|@\xbf\x93@\xdeJ\xb0\xc1[\xe8F\xd0C\xdc2\x7f{M\x9b\\7Z\x93\x80S\x882\xec\xa8\xdbpa\x98\xdc\xe6\x7f]\xb3\rp\x97O}\xb8 \x17\xee\x10\xee\xb82B\xe0\xcf\xeehq\x9a~"\x17\x18]\xe5te\xc6\xdd.\xee\xd0w)\xb9hq\x85\xb0-K\x90\x8f\xab\xb2\xe8\x9c2c\x03C\x9aO\x8d~\x1b\x8f\x98\x1c,\\\xf0.\xba\x88\x9a\xa3\x1d{\x82\x9a\xf3\x86P2[C\x0b\xb0<\xed[\xbf\xad\xd7tL\x91\x15P\x84\x0e\x9a\xa8\x8bq\xe6!\xc7\xc6P\xaa\x06\xfd\x1e6\xae \xb6KV\x89\xedo\xe5\x91\xf6\xfb\xc4\xe9>\xd0\x17\xbd\x15S\xd1\xef\xb7/\xd9d\x02Cf>;\x85\xc8MA\x10\xa5\x85=\x00\x00\x01\x00cc.\x0c\xf0u\x82mu\xd4\xd6q<\xb3\x0c\xbe\xd2\x0b\x80\xe8\xb3\x1dK\xc0\x82\x13h\xf7u\xa2\xc3j\xfa\x19\xa4\xff\x94\xc8L\xf3\xe5\xe3\x05\xfbl\xd4\x86\xb4\x81\xe5:\xda\xfa\xe15`W\x06\xf7y\xe8\x16#\x83@\xd2\xdb5\xc9\xf4W\xe6\x1bxm\x1f,\